In [2]:
import pandas as pd

In [10]:
df = pd.read_csv("./data/파일이름.csv", encoding='utf-8', index_col=0)
df = pd.read_csv("./data/coupang.csv", encoding='utf-8', index_col=0)

/tmp/ipykernel_28064/4184602919.py:1: DtypeWarning: Columns (21,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/coupang.csv", encoding='utf-8', index_col=0)


In [14]:
df = df.drop_duplicates('본문').reset_index(drop = True)

추가적으로 민원을 통쨰로 제거할 필요가 있는 경우 \
이 부분에 코드를 추가하세요

In [16]:
text = df['본문']

In [17]:
text

0        현 국민연금 가입내역 안내서 에서 일용직 아르바이트 쿠팡문류센터에서 근무한기록이 있...
1        최근 에 제 상황들을 한국 서울 국가인권위원회에서 \r\n\r\n국비지원진로준비들중...
2        1993 년생 윤은주 라고 합니다 .\r\n최근에 저나 제 식구들을\r\n보호하거나...
3        안녕하세요~\r\n\r\n갑자기 추워진 날씨에 수고가 많으십니다. \r\n\r\n저...
4        관련자료\r\n\r\nhttps://www.youtube.com/watch?v=yb...
                               ...                        
23126    물류센터 반대 민원\r\n검단신도시 물류유통3 재검토 및 공청회 요청\r\n\r\n...
23127    이랜드몰에서 청소기를 샀는데요\n받아서 사용하는데 대가 휘어져있는거에요? 그리고 제...
23128    제가 쿠팡에서 아래 식품&#x28;육회&#x29;를 9.16&#x28;금&#x29;...
23129    우체국 쇼핑이고 농협이고 k쇼핑이고 다 반품 안된다고 하고요\n그래서 오미자주스 1...
23130    안녕하세요.\r\n\r\n저는 국민신문고를 통하여 방송통신기자재의 전파법 미준수한 ...
Name: 본문, Length: 23131, dtype: object

Tokenizing

토크나이징에는 konlpy의 Okt를 사용합니다. \
패키지가 없으면 다운로드 받으시고, 다른 pos tagger를 사용하고 싶으시면 수정하세요

In [19]:
from konlpy.tag import Okt

In [20]:
okt = Okt()

In [21]:
#민원에 양에 따라 시간이 조금 걸릴 수도 있습니다.
#민원의 길이가 1000글자 이상일 경우 토크나이징을 수행하지 않고 Too Long이라는 토큰으로 대치합니다.
tokens = text.apply(lambda x : okt.nouns(str(x)) if len(str(x)) < 1000 else "Too Long")